**<hr><h>Capstone Project: Webscrapping and Caniadian Postal Codes with Coordinates<h><hr>**

**Setup Environment: Import modules**

In [13]:
! pip install beautifulsoup4
import pandas as pd
import requests
from bs4 import BeautifulSoup
import urllib.request

**Scrapping dataset from web and setup pandas dataframe table**

In [14]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
res =  urllib.request.urlopen(url)
soup = BeautifulSoup(res,'html')
table = soup.find('table') 
table
A=[]
B=[]
C=[]
for row in table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))
df=pd.DataFrame(A,columns=['Postal Code'])
df['Borough']=B
df['Neighborhood']=C

**Data cleanup: Remove "Not assigned" Borough's and fill "Not assigned"Neighborhoods**

In [15]:
indexNames = df[df['Borough'] == 'Not assigned'].index #look through column to not assigned
df.drop(indexNames, inplace=True)#delete rows with not assisgned
df.Neighborhood.replace('Not assigned\n',df.Borough, inplace=True)

**Group PostalCode Neoghborhoods and compute new shape of dataframe**

In [16]:
df_pc = df.groupby(by=['Postal Code','Borough'],sort = False)['Neighborhood'].agg(', '.join).reset_index()

**Import Coordinates from CSV file**

In [23]:
df_gcoder = pd.read_csv('http://cocl.us/Geospatial_data', index_col=0)

**Combine Neighborhood and Coordinates dataframes via pandas inner join**

In [24]:
df_merged=pd.merge(df_pc, df_gcoder, on='Postal Code', how='inner')
df_merged.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North\n,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson\n, Garden District\n",43.657162,-79.378937
